# Calculating total material amounts

This notebook shows how to calculate material amounts for each building or for the entire building stock, based on the iBuildGreen database. So far, we have associated buildings with macrocomponents, and macrocomponents with LCAbyg constructions and products. We know what component goes in what building, but we haven't quantified anything yet. This notebook is about dimensioning components and quantifying material amounts

# Setup

In [17]:
import psycopg as pg
import pandas as pd

In [18]:
params='dbname=macrocomponents user=postgres password=19Ni93co44PG!'

## Global dimensioning parameters

The parameters below are used in a first run of the model to estimate building dimensions (floor height, perimeter, etc) as we do not have precise information on building geometry at this stage.

In [20]:
FloorHeight = 3.5

In [21]:
WindowWallRatio = 0.2

We define a space efficiency factor as the ratio between the building's actual length, and its length if the building had been square, with the same floor area.

In [19]:
SEF = 1.2 

We also define an SQL snippet to estimate the building's perimeter based on the floor area and space efficiency factor. The perimeter will be used primarily to dimension external walls and windows. At this stage, the code does not handle well buildings that have floors of different sizes. Therefore, we approximate the perimeter using the total floor area divided by the number of floors, instead of using the footprint. For instance, if the building has a 600 m2 ground floor and a 200 m2 first floor, it will be treated as having two 400 m2 floors for the purpose of dimensioning the perimeter (otherwise, if we used the footprint, the dimensioning function later on would multiply 600 m2 by the number of floors and overestimate the size of the building).

This could be improved later on by using BBR data about Etages (building floors). However, since it requires importing and working with a completely different object, and linking Etages with Buildings, it requires a significant amount of work.

There are cases where the building's total floor area is missing. In such cases, the building is assumed to have one storey and a floor area equal to the footprint. This is implemented in the SQL snippet using the COALESCE function. COALESCE returns its first argument if the argument is not NULL, or its second argument otherwise.

In [28]:
get_perimeter_sql=f"SELECT COALESCE(SQRT(b.total_building_area/b.n_floors),SQRT(b.built_area))*2*(%s+1/%s) as perimeter" % (SEF,SEF) #Rough approximation if the building has storeys of different sizes

SELECT COALESCE(SQRT(b.total_building_area/b.n_floors),SQRT(b.built_area))*2*(1.2+1/1.2) as perimeter


## Estimating the surface of internal walls

As we do not have any information on internal walls from the BBR database, we must first estimate the surface of internal walls. In this first version of the model, research by Adélaide Mailhac on dimensioning functions for French buildings is used. This might entail an error if Danish buildings are architecturally very different from French buildings.

The surfaces of load-bearing and non-load-bearing internal walls are estimated separately based on the building's use type and its floor area. These estimated surfaces are reported in separate columns in the buildings table. First, we write SQL statements to estimate wall surfaces:

In [26]:
fill_int_walls_lb="""
WITH iws AS (
SELECT
b.bbr_id AS bbrid,
intwallsurface
FROM buildings b,
LATERAL (SELECT b.bbr_use_category::int4 AS use_code) lt1,
LATERAL (SELECT COALESCE(b.total_building_area,b.built_area) AS floor_area) lt2,
LATERAL (SELECT 
         (CASE 
    WHEN use_code IN (110, 120, 121, 122, 130, 131, 132, 325, 510, 520, 521, 522, 523, 529, 530, 540, 585, 590)
        THEN 0.222 * floor_area 
    WHEN use_code IN (140, 150, 160, 185, 190, 320, 321, 322, 324, 329, 390, 410, 411, 412, 413, 414, 415, 419, 531, 532, 533, 534, 539)
        THEN floor_area * (0.4063 + 0.00003489 * floor_area)
    WHEN use_code <@ int4range(420,490)
        THEN floor_area * (0.4063 + 0.00003489 * floor_area)
    WHEN use_code <@ int4range(210,319)
        THEN floor_area*0.1
    WHEN use_code IN (323, 416,535)
        THEN floor_area*0.1
    ELSE NULL
          END) AS intwallsurface) lt
)

INSERT INTO buildings (bbr_id,int_wall_surface_lb)
SELECT bbrid, intwallsurface FROM iws
ON CONFLICT ON CONSTRAINT buildings_pkey DO UPDATE SET (bbr_id,int_wall_surface_lb) = (EXCLUDED.bbr_id,EXCLUDED.int_wall_surface_lb)"""

The dimensioning of non-load-bearing internal walls uses a compacity index "icomp", calculated using the building's external surface (surface of external walls and roof) and its volume.

In [1]:
fill_int_walls_nlb="""
WITH iws AS (
SELECT
b.bbr_id AS bbrid,
intwallsurface
FROM buildings b,
LATERAL (SELECT b.bbr_use_category::int4 AS use_code) lt1,
LATERAL (SELECT COALESCE(b.total_building_area,b.built_area) AS floor_area) lt2,
LATERAL (SELECT %s*floor_area AS volume) lt3,
LATERAL (%s) lt4,
LATERAL (SELECT (CASE WHEN b.n_floors IS NOT NULL THEN b.built_area+perimeter*b.n_floors*%s ELSE b.built_area+perimeter*%s END) AS external_surface) lt5,
LATERAL (SELECT (CASE WHEN volume<=0 THEN NULL ELSE external_surface/POWER(volume,0.666667) END) AS icomp) lt6,
    LATERAL (SELECT 
         (CASE 
    WHEN use_code IN (110, 120, 121, 122, 130, 131, 132, 325, 510, 520, 521, 522, 523, 529, 530, 540, 585, 590)
        THEN 0.37 * floor_area 
    WHEN use_code IN (140, 150, 160, 185, 190, 320, 321, 322, 324, 329, 390, 410, 411, 412, 413, 414, 415, 419, 531, 532, 533, 534, 539)
        THEN floor_area * (0.1803 + 0.0883 * icomp)
    WHEN use_code <@ int4range(420,490)
        THEN floor_area * (0.1803 + 0.0883 * icomp)
    WHEN use_code <@ int4range(210,319)
        THEN floor_area*0.15
    WHEN use_code IN (323, 416,535)
        THEN floor_area*0.15
    ELSE NULL
          END) AS intwallsurface) lt
)

INSERT INTO buildings (bbr_id,int_wall_surface_nlb)
SELECT bbrid, intwallsurface FROM iws
ON CONFLICT ON CONSTRAINT buildings_pkey DO UPDATE SET (bbr_id,int_wall_surface_nlb) = (EXCLUDED.bbr_id,EXCLUDED.int_wall_surface_nlb)
""" % (FloorHeight,get_perimeter_sql,FloorHeight,FloorHeight)

Then, we use a simple function to compute these SQL statements and insert values for internal wall surfaces in the database.

In [30]:
def fill_int_walls_amounts():
    try:
        conn = pg.connect(params)
        cur=conn.cursor()
        
        cur.execute(fill_int_walls_lb)
        cur.execute(fill_int_walls_nlb)
       
        conn.commit()
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()  

In [31]:
fill_int_walls_amounts()

# Getting material amounts for all elements

This section shows how to quantify material amounts for all components of all buildings in the database. These results are reported in the "results_material_amounts" table in the database. 

First, we define SQL queries to dimension material amounts for each type of component.

In [42]:
result_ext_wall_sql="""
WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    perimeter,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_ext_walls bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN ext_wall_types type
    ON bmap.ext_wall_id=type.id
INNER JOIN ext_walls_to_subcomponents submap
    ON type.id = submap.ext_wall_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (%s) ltp,
LATERAL (SELECT (CASE WHEN b.n_floors IS NOT NULL THEN perimeter*b.n_floors*%s*(1-%s)*pmap.amount ELSE perimeter*%s*(1-%s)*pmap.amount END) AS amount_product) lta)

INSERT INTO results_material_amounts(element, product, amount, unit, bbr_id)
SELECT 'ext_wall', product, amount_product, unit, bbrid
FROM quant_table
""" % (get_perimeter_sql, FloorHeight, WindowWallRatio, FloorHeight, WindowWallRatio)

In [43]:
result_window_sql="""
WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    sc.name,
    perimeter,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN subcomponents sc ON sc.name = 'Window - iBuildGreen'
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (%s) ltp,
LATERAL (SELECT (CASE WHEN b.n_floors IS NOT NULL THEN perimeter*b.n_floors*%s*%s*pmap.amount ELSE perimeter*%s*%s*pmap.amount END) AS amount_product) lta)

INSERT INTO results_material_amounts(element, product, amount, unit, bbr_id)
SELECT 'window', product, amount_product, unit, bbrid
FROM quant_table
""" % (get_perimeter_sql, FloorHeight, WindowWallRatio, FloorHeight, WindowWallRatio)

In [44]:
result_int_wall_sql="""WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_int_walls bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN int_wall_types type
    ON bmap.int_wall_id=type.id
INNER JOIN int_walls_to_subcomponents submap
    ON type.id = submap.int_wall_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT (b.int_wall_surface_nlb+b.int_wall_surface_lb)*pmap.amount AS amount_product) lta)

INSERT INTO results_material_amounts(element,product,amount,unit, bbr_id)
SELECT 'int_wall',product,amount_product, unit, bbrid
FROM quant_table
"""

In [45]:
result_roof_cover_sql="""WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    roof_surface,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_roof_covers bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN roof_cover_types typ
    ON bmap.roof_cover_id=typ.id
INNER JOIN roof_covers_to_subcomponents submap
    ON typ.id = submap.roof_cover_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT b.built_area/COS(b.roof_pitch*PI()*180) AS roof_surface) lt1,
LATERAL (SELECT roof_surface*pmap.amount AS amount_product) lt2)

INSERT INTO results_material_amounts(element,product,amount,unit, bbr_id)
SELECT 'roof_cover',product,amount_product, unit, bbrid
FROM quant_table
"""

In [46]:
result_roof_structure_sql="""WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    typ.name,
    b.roof_pitch,
    roof_surface,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_roof_structures bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN roof_structure_types typ
    ON bmap.roof_structure_id=typ.id
INNER JOIN roof_structures_to_subcomponents submap
    ON typ.id = submap.roof_structure_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT b.built_area/COS(b.roof_pitch*PI()*180) AS roof_surface) lt1,
LATERAL (SELECT roof_surface*pmap.amount AS amount_product) lt2
WHERE typ.name <> 'Ridge board')

INSERT INTO results_material_amounts(element,product,amount,unit, bbr_id)
SELECT 'roof_structure',product,amount_product, unit, bbrid
FROM quant_table
"""

In [47]:
result_floor_sql="""WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_floors bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN floor_types type
    ON bmap.floor_id=type.id
INNER JOIN floors_to_subcomponents submap
    ON type.id = submap.floor_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT COALESCE(b.total_building_area,b.built_area)*pmap.amount AS amount_product) lta)

INSERT INTO results_material_amounts(element,product,amount,unit, bbr_id)
SELECT 'floor',product,amount_product, unit, bbrid
FROM quant_table
"""

In [48]:
result_foundation_sql="""WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_foundations bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN foundation_types type
    ON bmap.foundation_id=type.id
INNER JOIN foundations_to_subcomponents submap
    ON type.id = submap.foundation_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT b.built_area*pmap.amount AS amount_product) lta)

INSERT INTO results_material_amounts(element,product,amount,unit, bbr_id)
SELECT 'foundation',product,amount_product, unit, bbrid
FROM quant_table
"""

In [49]:
result_ridge_board_sql="""WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    b.built_area,
    typ.name,
    b.roof_pitch,
    beam_length,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_roof_structures bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN roof_structure_types typ
    ON bmap.roof_structure_id=typ.id
INNER JOIN roof_structures_to_subcomponents submap
    ON typ.id = submap.roof_structure_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT SQRT(b.built_area)*%s AS beam_length) lt1,
LATERAL (SELECT beam_length*pmap.amount AS amount_product) lt2
WHERE typ.name = 'Ridge board')

INSERT INTO results_material_amounts(element,product,amount,unit, bbr_id)
SELECT 'ridge_board',product,amount_product, unit, bbrid
FROM quant_table
""" % SEF

In [50]:
def fill_buildings_amounts_table():
    try:
        conn = pg.connect(params)
        cur=conn.cursor()
        
        cur.execute("DELETE FROM results_material_amounts")
        cur.execute(result_ext_wall_sql)
        cur.execute(result_int_wall_sql)
        cur.execute(result_roof_structure_sql)
        cur.execute(result_roof_cover_sql)
        cur.execute(result_floor_sql)
        cur.execute(result_foundation_sql)
        cur.execute(result_ridge_board_sql)
        cur.execute(result_window_sql)

       
        conn.commit()
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()  

In [51]:
fill_buildings_amounts_table()

Alternatively, we can define SQL queries to directly retrieve the total amounts of each type of material, in each type of element:

In [32]:
tot_ext_wall_sql="""
WITH quant_table AS
(SELECT 
    perimeter,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_ext_walls bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN ext_wall_types type
    ON bmap.ext_wall_id=type.id
INNER JOIN ext_walls_to_subcomponents submap
    ON type.id = submap.ext_wall_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (%s) ltp,
LATERAL (SELECT (CASE WHEN b.n_floors IS NOT NULL THEN perimeter*b.n_floors*%s*(1-%s)*pmap.amount ELSE perimeter*%s*(1-%s)*pmap.amount END) AS amount_product) lta)

INSERT INTO tot_material_amounts(element, product, amount, unit)
SELECT 'ext_wall', product, SUM(amount_product), unit
FROM quant_table
GROUP BY product, unit""" % (get_perimeter_sql, FloorHeight, WindowWallRatio, FloorHeight, WindowWallRatio)

In [33]:
tot_window_sql="""
WITH quant_table AS
(SELECT 
    b.bbr_id,
    sc.name,
    perimeter,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN subcomponents sc ON sc.name = 'Window - iBuildGreen'
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (%s) ltp,
LATERAL (SELECT (CASE WHEN b.n_floors IS NOT NULL THEN perimeter*b.n_floors*%s*%s*pmap.amount ELSE perimeter*%s*%s*pmap.amount END) AS amount_product) lta)

INSERT INTO tot_material_amounts(element, product, amount, unit)
SELECT 'window', product, SUM(amount_product), unit
FROM quant_table
GROUP BY product, unit""" % (get_perimeter_sql, FloorHeight, WindowWallRatio, FloorHeight, WindowWallRatio)

In [34]:
tot_int_wall_sql="""WITH quant_table AS
(SELECT 
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_int_walls bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN int_wall_types type
    ON bmap.int_wall_id=type.id
INNER JOIN int_walls_to_subcomponents submap
    ON type.id = submap.int_wall_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT (b.int_wall_surface_nlb+b.int_wall_surface_lb)*pmap.amount AS amount_product) lta)

INSERT INTO tot_material_amounts(element,product,amount,unit)
SELECT 'int_wall',product,SUM(amount_product),unit
FROM quant_table
GROUP BY product, unit"""

In [35]:
tot_roof_cover_sql="""WITH quant_table AS
(SELECT 
    roof_surface,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_roof_covers bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN roof_cover_types typ
    ON bmap.roof_cover_id=typ.id
INNER JOIN roof_covers_to_subcomponents submap
    ON typ.id = submap.roof_cover_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT b.built_area/COS(b.roof_pitch*PI()*180) AS roof_surface) lt1,
LATERAL (SELECT roof_surface*pmap.amount AS amount_product) lt2)

INSERT INTO tot_material_amounts(element,product,amount,unit)
SELECT 'roof_cover',product,SUM(amount_product),unit
FROM quant_table
GROUP BY product, unit"""

In [36]:
tot_roof_structure_sql="""WITH quant_table AS
(SELECT 
    typ.name,
    b.roof_pitch,
    roof_surface,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_roof_structures bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN roof_structure_types typ
    ON bmap.roof_structure_id=typ.id
INNER JOIN roof_structures_to_subcomponents submap
    ON typ.id = submap.roof_structure_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT b.built_area/COS(b.roof_pitch*PI()*180) AS roof_surface) lt1,
LATERAL (SELECT roof_surface*pmap.amount AS amount_product) lt2
WHERE typ.name <> 'Ridge board')


INSERT INTO tot_material_amounts(element,product,amount,unit)
SELECT 'roof_structure',product,SUM(amount_product),unit
FROM quant_table
GROUP BY product, unit"""

In [37]:
tot_floor_sql="""WITH quant_table AS
(SELECT 
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_floors bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN floor_types type
    ON bmap.floor_id=type.id
INNER JOIN floors_to_subcomponents submap
    ON type.id = submap.floor_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT b.total_building_area*pmap.amount AS amount_product) lta)

INSERT INTO tot_material_amounts(element,product,amount,unit)
SELECT 'floor',product,SUM(amount_product),unit
FROM quant_table
GROUP BY product, unit"""

In [38]:
tot_foundation_sql="""WITH quant_table AS
(SELECT 
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_foundations bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN foundation_types type
    ON bmap.foundation_id=type.id
INNER JOIN foundations_to_subcomponents submap
    ON type.id = submap.foundation_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT b.built_area*pmap.amount AS amount_product) lta)

INSERT INTO tot_material_amounts(element,product,amount,unit)
SELECT 'foundation',product,SUM(amount_product),unit
FROM quant_table
GROUP BY product, unit"""

In [39]:
tot_ridge_board_sql="""WITH quant_table AS
(SELECT 
    b.bbr_id bbrid,
    b.built_area,
    typ.name,
    b.roof_pitch,
    beam_length,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_roof_structures bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN roof_structure_types typ
    ON bmap.roof_structure_id=typ.id
INNER JOIN roof_structures_to_subcomponents submap
    ON typ.id = submap.roof_structure_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (SELECT SQRT(b.built_area)*%s AS beam_length) lt1,
LATERAL (SELECT beam_length*pmap.amount AS amount_product) lt2
WHERE typ.name = 'Ridge board')


INSERT INTO tot_material_amounts(element,product,amount,unit)
SELECT 'ridge_board',product,SUM(amount_product),unit
FROM quant_table
GROUP BY product, unit""" % SEF

In [40]:
def fill_total_amounts_table():
    try:
        conn = pg.connect(params)
        cur=conn.cursor()
        
        cur.execute("DELETE FROM tot_material_amounts")
        cur.execute(tot_ext_wall_sql)
        cur.execute(tot_int_wall_sql)
        cur.execute(tot_roof_structure_sql)
        cur.execute(tot_roof_cover_sql)
        cur.execute(tot_floor_sql)
        cur.execute(tot_foundation_sql)
        cur.execute(tot_ridge_board_sql)
        cur.execute(tot_window_sql)

       
        conn.commit()
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()  

In [41]:
fill_total_amounts_table()

### Retrieving info for one building

In [52]:
import pandas as pd

In [53]:
def results_one_building(bbr_id):
    results=pd.DataFrame({'bbr_id':[],'element':[],'product':[],'weight':[],'material_type':[]})
    
    SQL="""
    SELECT
    rma.bbr_id,
    rma.element,
    rma.product,
    weight,
    material_type
    FROM results_material_amounts rma
    INNER JOIN products pr ON rma.product=pr.name,
    LATERAL (SELECT (CASE WHEN unit='KG' THEN amount WHEN unit='M3' THEN amount*pr.density ELSE NULL END) AS weight) lt
    WHERE rma.bbr_id=%s
    ORDER BY element  
    """
    
    try:
        conn = pg.connect(params)
        cur=conn.cursor()
        
        cur.execute(SQL,(bbr_id,))
        row=cur.fetchone()     
              
        while row is not None:
            results.loc[len(results)]=list(row)
            row=cur.fetchone()

        conn.commit()
        cur.close()
        
        return(results)

    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close() 

In [54]:
def properties_one_building(bbr_id):
    results=pd.DataFrame({'bbr_id':[],'construction_year':[],'bbr_use_category':[],'built_area':[], 'total_building_area':[], 'n_floors':[], 'roof_material':[], 'wall_material':[]})
    
    SQL="""
    SELECT
    b.bbr_id,
    b.construction_year,
    b.bbr_use_category,
    b.built_area,
    b.total_building_area,
    b.n_floors,
    b.roof_material,
    b.wall_material
    FROM buildings b
    WHERE bbr_id=%s
    """
    
    try:
        conn = pg.connect(params)
        cur=conn.cursor()
        
        cur.execute(SQL,(bbr_id,))
        row=cur.fetchone()     
              
        while row is not None:
            results.loc[len(results)]=list(row)
            row=cur.fetchone()

        conn.commit()
        cur.close()
        
        return(results)

    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()    


In [55]:
def material_amounts_one_building(bbr_id):
    results=pd.DataFrame({'bbr_id':[],'clay':[],'cement_mortar':[],'concrete':[],'gypsum_plaster':[], 'metal':[],'wood':[],'wool':[],'other':[]})
    
    SQL1="""
    CREATE TEMP TABLE IF NOT EXISTS agg_material (
        bbr_id varchar(50) PRIMARY KEY,
        Clay real,
        Cement_mortar real,
        Concrete real,
        Gypsum_plaster real,
        Metal real,
        Wood real,
        Wool real,
        Other real);"""
    
    SQL2="""
    WITH t AS(
    SELECT
    rma.bbr_id,
    rma.element,
    rma.product,
    weight,
    material_type
    FROM results_material_amounts rma
    INNER JOIN buildings b ON b.bbr_id=rma.bbr_id
    INNER JOIN products pr ON rma.product=pr.name,
    LATERAL (SELECT (CASE WHEN unit='KG' THEN amount WHEN unit='M3' THEN amount*pr.density ELSE NULL END) AS weight) lt
    WHERE rma.bbr_id=%s
    )

    SELECT
    bbr_id,
    SUM(weight) weightsum,
    material_type
    INTO TEMPORARY TABLE t2
    FROM t
    GROUP BY material_type, bbr_id;    
    """

    SQLclay="""
    INSERT INTO agg_material(bbr_id, Clay)
    SELECT 
    bbr_id,
    weightsum
    FROM t2
    WHERE material_type = 'Clay'
    ON CONFLICT ON CONSTRAINT agg_material_pkey DO UPDATE SET (bbr_id, Clay) = (EXCLUDED.bbr_id, EXCLUDED.Clay);
    """
    
    SQLcement="""
    INSERT INTO agg_material(bbr_id, Cement_mortar)
    SELECT 
    bbr_id,
    weightsum
    FROM t2
    WHERE material_type = 'Cement-mortar' 
    ON CONFLICT ON CONSTRAINT agg_material_pkey DO UPDATE SET (bbr_id, Cement_mortar) = (EXCLUDED.bbr_id, EXCLUDED.Cement_mortar);
    """

    SQLconcrete="""
    INSERT INTO agg_material(bbr_id, Concrete)
    SELECT 
    bbr_id,
    weightsum
    FROM t2
    WHERE material_type = 'Concrete'
    ON CONFLICT ON CONSTRAINT agg_material_pkey DO UPDATE SET (bbr_id, Concrete) = (EXCLUDED.bbr_id, EXCLUDED.Concrete);
    """

    SQLgypsum="""
    INSERT INTO agg_material(bbr_id, Gypsum_plaster)
    SELECT 
    bbr_id,
    weightsum
    FROM t2
    WHERE material_type = 'Gypsum-plaster'
    ON CONFLICT ON CONSTRAINT agg_material_pkey DO UPDATE SET (bbr_id, Gypsum_plaster) = (EXCLUDED.bbr_id, EXCLUDED.Gypsum_plaster);
    """

    SQLmetal="""
    INSERT INTO agg_material(bbr_id, Metal)
    SELECT 
    bbr_id,
    weightsum
    FROM t2
    WHERE material_type = 'Metal'
    ON CONFLICT ON CONSTRAINT agg_material_pkey DO UPDATE SET (bbr_id, Metal) = (EXCLUDED.bbr_id, EXCLUDED.Metal);
    """

    SQLwood="""
    INSERT INTO agg_material(bbr_id, Wood)
    SELECT 
    bbr_id,
    weightsum
    FROM t2
    WHERE material_type = 'Wood'
    ON CONFLICT ON CONSTRAINT agg_material_pkey DO UPDATE SET (bbr_id, Wood) = (EXCLUDED.bbr_id, EXCLUDED.Wood);
    """

    SQLwool="""
    INSERT INTO agg_material(bbr_id, Wool)
    SELECT 
    bbr_id,
    weightsum
    FROM t2
    WHERE material_type = 'Wool'
    ON CONFLICT ON CONSTRAINT agg_material_pkey DO UPDATE SET (bbr_id, Wool) = (EXCLUDED.bbr_id, EXCLUDED.Wool);
    """

    SQLother="""
    INSERT INTO agg_material(bbr_id, Other)
    SELECT 
    bbr_id,
    weightsum
    FROM t2
    WHERE material_type = 'Other'
    ON CONFLICT ON CONSTRAINT agg_material_pkey DO UPDATE SET (bbr_id, Other) = (EXCLUDED.bbr_id, EXCLUDED.Other);
    """

    SQLselect="SELECT * FROM agg_material"
    
    try:
        conn = pg.connect(params)
        cur=conn.cursor()
        
        cur.execute(SQL1)
        cur.execute(SQL2,(bbr_id,))
        cur.execute(SQLclay)
        cur.execute(SQLcement)
        cur.execute(SQLconcrete)
        cur.execute(SQLgypsum)
        cur.execute(SQLmetal)
        cur.execute(SQLwood)
        cur.execute(SQLwool)
        cur.execute(SQLother)
        cur.execute(SQLselect)       

        row=cur.fetchone()
        
        while row is not None:
            results.loc[len(results)]=list(row)
            row=cur.fetchone()

        conn.commit()
        cur.close()
        
        return(results)

    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close() 

In [ ]:
results_one_building('7A2AF1D0-CE0C-4784-BB3F-1054A34C4F60'.lower())

In [256]:
properties_one_building('7A2AF1D0-CE0C-4784-BB3F-1054A34C4F60'.lower())

,bbr_id,construction_year,bbr_use_category,built_area,total_building_area,n_floors,roof_material,wall_material
0,7a2af1d0-ce0c-4784-bb3f-1054a34c4f60,2002,220,1369,1369,1,3,6


In [221]:
material_amounts_one_building('7A2AF1D0-CE0C-4784-BB3F-1054A34C4F60'.lower())

,bbr_id,clay,cement_mortar,concrete,gypsum_plaster,metal,wood,wool,other
0,7a2af1d0-ce0c-4784-bb3f-1054a34c4f60,None,26327.227,1287358.2,22167.934,34943.664,37192.27,7595.6685,1369.0


# Computing MICs